In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

df = pd.read_csv("D:/kaggle/titanic/train.csv", index_col="PassengerId")
df.head()

# DataPreprocessing
l, b = df.shape

# 缺失值处理
# 判断一列中是否有缺失值：np.all(df["Pclass"].notnull())
df["Age"].fillna(df["Age"].median(), inplace=True)
df["Cabin"].fillna("ab", inplace=True)
df["Embarked"].fillna("S", inplace=True)

df["Sex"] = df["Sex"].replace({"male":str(1), "female":str(0)}).astype(int)

In [60]:
from sklearn.preprocessing import LabelEncoder    
le = LabelEncoder()  # 将一组离散型(n类)的数据转换成0~n-1之间的数
df["Embarked"] = le.fit_transform(df["Embarked"])

In [69]:
# 移除离群值（outliers）
q = df["Fare"].quantile(0.99)
df2 = df[df["Fare"]<q]
tar = df2['Survived']
fea=df2.drop(["Survived","Name","Ticket","Cabin"],axis=1)
fea

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,3,1,22.0,1,0,7.2500,2
2,1,0,38.0,1,0,71.2833,0
3,3,0,26.0,0,0,7.9250,2
4,1,0,35.0,1,0,53.1000,2
5,3,1,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...
887,2,1,27.0,0,0,13.0000,2
888,1,0,19.0,0,0,30.0000,2
889,3,0,28.0,1,2,23.4500,2


In [70]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(fea, tar, test_size=0.3, random_state=42, shuffle=True)

In [72]:
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score
from xgboost import XGBClassifier
reg = XGBClassifier()

In [74]:
params={"n_estimators":[67,70,100,120],'reg_lambda':[2,1],'gamma':[0,0.3,0.2,0.1]
       ,'eta':[0.06,0.05,0.04]
        ,"max_depth":[3,5],'objective':['binary:logistic']}

In [77]:
clf=GridSearchCV(reg,params,cv=10,n_jobs=-1,verbose=1)
clf.fit(x_train,y_train)

Fitting 10 folds for each of 192 candidates, totalling 1920 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   24.1s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   50.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 1920 out of 1920 | elapsed:  3.2min finished
E:\anaconda\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
E:\anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, 

[19:02:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constra...
                                     subsample=None, tree_method=None,
                                     use_label_encoder=True,
                                     validate_parameters=None, verbosity=None),
             iid='warn', n_jobs=-1,
             param_grid={'eta': [0.06, 0.05, 0.04], 'gamma': [0, 0.3, 0.2

In [78]:
ypred=clf.predict(x_train)
tpred=clf.predict(x_test)
print(accuracy_score(ypred,y_train))
print(accuracy_score(tpred,y_test))
clf.best_params_

0.8962722852512156
0.8452830188679246


{'eta': 0.06,
 'gamma': 0.3,
 'max_depth': 5,
 'n_estimators': 120,
 'objective': 'binary:logistic',
 'reg_lambda': 2}

In [79]:
tes=pd.read_csv("D:/kaggle/titanic/test.csv",index_col="PassengerId")
tes

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S


In [80]:
tes["Age"]=tes["Age"].fillna(tes["Age"].median())
tes["Sex"]=le.fit_transform(tes["Sex"])
tes["Embarked"]=le.fit_transform(tes["Embarked"])
tes["Fare"]=tes["Fare"].fillna(tes["Fare"].median())
tes.isnull().sum()
tfea=tes.drop(["Name","Ticket","Cabin"],axis=1)
my_pred=clf.predict(tfea)
submit=pd.DataFrame(tes.index,columns=["PassengerId"])
submit["Survived"]=my_pred
submit

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [82]:
submit.to_csv("D:/predict_titanic.csv", index=False)